### HEMOCE 

Dada uma sequência de números ($ 0, 1, 2, ... N $) chegando em tempo real. Você precisa calcular a soma desta sequência.
Use uma abordagem sensata para armazenar e atualizar o resultado atual.

A saída deve ser um número.

**Exemplo**
* Sequência de entrada: `0, 1, 2, 3`.
* Saída: `6`


In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.0-preview/spark-3.0.0-preview-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-preview-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview-bin-hadoop2.7"
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType
from time import sleep
from pyspark.streaming import StreamingContext
#Spark Contexto
#df = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

from pyspark.sql import SparkSession

ss = SparkSession \
    .builder \
    .appName("Python Spark create Streaming example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
    
sc.version 

'3.0.0-preview'

In [0]:
from time import sleep
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

#sc = SparkContext(master='local[4]')

NUM_BATCHES = 4  # a quantidade de números em sequência
batches = [sc.parallelize([num]) for num in range(NUM_BATCHES)]

BATCH_TIMEOUT = 5 # Tempo limite entre geração de lote
ssc = StreamingContext(sc, BATCH_TIMEOUT)
dstream = ssc.queueStream(rdds=batches)

In [0]:
finished = False
printed = False

def set_ending_flag(rdd): # Seta o final do flag
    global finished
    if rdd.isEmpty():
        finished = True

def print_only_at_the_end(rdd): #  imprimir apenas no final
    global printed
    if finished and not printed:
        print(rdd.collect()[0])
        printed = True

# Se recebermos o rdd vazio, o fluxo será concluído.
# Então imprima o resultado e pare o contexto.

dstream.foreachRDD(set_ending_flag)

In [0]:
def aggregator(values, old):
    return (old or 0) + sum(values)

# `updateStateByKey` precisa da estrutura de valores-chave, você precisa especificar a chave fictícia" res "
# e remova-o após a agregação

dstream.map(lambda num: ('res', num))\
    .updateStateByKey(aggregator)\
    .map(lambda x: x[1])\
    .foreachRDD(print_only_at_the_end)

In [0]:
ssc.checkpoint('./checkpoint')  #  ponto de verificação para armazenar o estado atual
ssc.start()
while not printed:
    pass
ssc.stop()                      #  Quando ele conta o tempo de NUM_BATCHES ele finaliza
sc.stop()

6
